## requirements

In [1]:
%%capture

!pip install diffusers["torch"] transformers
!pip install gradio

## processing.py

In [2]:
%%capture

import scipy
import torch
from diffusers import AudioLDMPipeline
from diffusers import AudioLDM2Pipeline

In [5]:
%%capture

# processing
def processing(prompt:str, name_file: str, model:str):

    if model == "good":
      repo_id = "cvssp/audioldm-s-full-v2"
      pipe = AudioLDMPipeline.from_pretrained(repo_id, torch_dtype=torch.float16)
      pipe = pipe.to("cuda")

      prompt = prompt
      audio = pipe(prompt, num_inference_steps=100, audio_length_in_s=5).audios[0]

      scipy.io.wavfile.write(f'{name_file}.wav', rate=16000, data=audio)

    elif model == "better":
      repo_id = "cvssp/audioldm2"
      pipe = AudioLDM2Pipeline.from_pretrained(repo_id, torch_dtype=torch.float16)
      pipe = pipe.to("cuda")

      prompt = prompt
      negative_prompt = "Low quality."

      generator = torch.Generator("cuda").manual_seed(0)

      audio = pipe(
          prompt,
          negative_prompt=negative_prompt,
          num_inference_steps=200,
          audio_length_in_s=10.0,
          num_waveforms_per_prompt=3,
          generator=generator
          ).audios

      scipy.io.wavfile.write(f'{name_file}.wav', rate=16000, data=audio[0])

    audio_file = f'{name_file}.wav'
    return audio_file


## app.py

In [38]:
import gradio as gr

gui = gr.Interface(processing,
 [
     gr.Textbox(label="Your prompt", info="digite qual tipo de som gostaria de criar?"),
     gr.Textbox(label="Como será o nome do seu arquivo?", info="escolha um nome para o seu arquivo!"),
     gr.Radio(["good", "better"], label="Modelo", info="qual modelo gostaria de utilizar nessa tarefa?")
     ],
      theme = gr.themes.Soft(),
      examples=[
        ["storm and thunder", "aiMusic", "better"],
    ],
      outputs="audio")

gui.launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://7f8ee7a53c3ed2c79f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading pipeline components...:   0%|          | 0/11 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://7f8ee7a53c3ed2c79f.gradio.live
